<a href="https://colab.research.google.com/github/bijuthottathil/azure/blob/main/DeltaTablePysparkMergeInCollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# setting up Pyspark in Colab
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark


In [41]:
# Setting up Delta table in Colab
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()
from delta.tables import *

In [42]:
# Reading Source File
source=spark.read.csv("/content/scd1source.csv", header=True)

In [43]:
# Creating Delta Table with partitions in Hive Meta store
source.write.format("delta").partitionBy("id").mode("overwrite").saveAsTable("source")

In [44]:
deltasource= DeltaTable.forPath(spark,"/content/spark-warehouse/source")

In [45]:
# Displaying Source Table Data
deltasource.toDF().show()

+---+--------+--------+
| id|    name|location|
+---+--------+--------+
|  3|Vikranth|   India|
|  2|    Mike| Germany|
|  1|    Jack|   Italy|
+---+--------+--------+



In [46]:
# Reading Destination File
dest=spark.read.csv("/content/scd1dest.csv", header=True)

In [47]:
# Creating Destination Delta Table with partitions in Hive Meta store
dest.write.format("delta").partitionBy("id").mode("overwrite").saveAsTable("dest")

In [48]:
deltadest= DeltaTable.forPath(spark,"/content/spark-warehouse/dest")

In [55]:
# Showing destinatin table data before Merging Source Table Data
dfdest=spark.sql("select * from dest order by id")
dfdest.show()

+---+--------+---------+
| id|    name| location|
+---+--------+---------+
|  1|    Jack|    Italy|
|  2|    Mike|  Germany|
|  3|Vikranth|    India|
|  4|     Raj|Hyderabad|
|  5|  Prasad|     Pune|
+---+--------+---------+



In [56]:
# Execute Merge Statement
mergesql=spark.sql("MERGE INTO dest  as t " + 
"USING source as  s on t.id=s.id " +
"WHEN MATCHED THEN  UPDATE SET * WHEN NOT MATCHED THEN INSERT * ")



In [57]:
# Result after SCD Type 1 Merge
dfdest=spark.sql("select * from dest order by id ")
dfdest.show()

+---+--------+---------+
| id|    name| location|
+---+--------+---------+
|  1|    Jack|    Italy|
|  2|    Mike|  Germany|
|  3|Vikranth|    India|
|  4|     Raj|Hyderabad|
|  5|  Prasad|     Pune|
+---+--------+---------+

